In [1]:
import flika_JSON_io as flikaIO
from pathlib import Path
import pandas as pd

## This notebook was used to take SVM4 classified JSON outputs for each condition and generate trajectory class specific .csv's for import into the MLE function by Bullerjahn & Hummer 2020.

In [16]:
def gen_single_condition_df(input_dir, inputf_mob, exp_label, pixel_width=0.1092):
    mobile = (Path(input_dir) / inputf_mob)

    file_list = [mobile]
    file_labels = ['mobile']

    dfs = {}
    for i, indiv_file in enumerate(file_list):
        temp_df = flikaIO.json_to_pandas([indiv_file], [exp_label])
        temp_df['TrajClass'] = file_labels[i]
        temp_df['X'] *= pixel_width
        temp_df['Y'] *= pixel_width
        # Choose the first 200 frames
        temp_df = temp_df.loc[temp_df['Frame'] <= 200]
        # Select every other frame
        temp_df = temp_df.iloc[::2, :]
        # write to file here
        outfile = Path(input_dir) / f"{exp_label}_{file_labels[i]}_0-200frames_stride2.csv"
        temp_df.to_csv(outfile, sep=',', index=False)
        dfs[file_labels[i]] = temp_df
    df = pd.concat(dfs, axis=0).reset_index(drop=True)
    return df

In [18]:
exp_labels = ['tdTomato_37Degree', 'tdTomato_Degree37_Endothelials', 'tdTomato_Degree37_mNSPCs']

for exp_label in exp_labels:
    if exp_label == 'tdTomato_37Degree':
        track_counts = [3158]
    if exp_label == 'tdTomato_Degree37_Endothelials':
        track_counts = [22682]
    if exp_label == 'tdTomato_Degree37_mNSPCs':
        track_counts = [190]

    input_dir = r"C:\\Users\User\\OneDrive\\Documents\\Python Programs\\Piezo1_MLE\\Trajectory_JSONs\\SVMv7_3CellTypes_Trajectory Data\\" + exp_label + r"\\Mobile\\"
    infile_mobile = f'SVMv7_predTrajs_{exp_label}_{str(track_counts[0])}Tracks_15SEP2021_Mobile.json'
    # This function also generates the .csv files in the input directories
    tdTomato_data = gen_single_condition_df(input_dir, infile_mobile, exp_label)